In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


/content/drive/MyDrive/SuperAI/Project/GDA/images

# Convert YOLO 1.1 format to YOLO-OBB format

In [ ]:
import os

def convert_aabb_to_obb(aabb_labels_dir, obb_labels_dir):
    # Ensure the OBB directory exists
    os.makedirs(obb_labels_dir, exist_ok=True)

    for label_file in os.listdir(aabb_labels_dir):
        aabb_path = os.path.join(aabb_labels_dir, label_file)
        obb_path = os.path.join(obb_labels_dir, label_file)

        with open(aabb_path, 'r') as f:
            lines = f.readlines()

        with open(obb_path, 'w') as f:
            for line in lines:
                class_id, x_center, y_center, width, height = map(float, line.split())
                # Convert center, width, height to x1, y1, x2, y2, x3, y3, x4, y4
                # Assuming no rotation, the OBB is the same as the AABB
                x1 = x_center - width / 2
                y1 = y_center - height / 2
                x2 = x_center + width / 2
                y2 = y_center - height / 2
                x3 = x_center + width / 2
                y3 = y_center + height / 2
                x4 = x_center - width / 2
                y4 = y_center + height / 2
                # Write the converted annotation to the OBB label file with spaces
                f.write(f"{int(class_id)} {x1} {y1} {x2} {y2} {x3} {y3} {x4} {y4}\n")

# Define your directories
aabb_labels_dir = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_labels'
obb_labels_dir = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels'

convert_aabb_to_obb(aabb_labels_dir, obb_labels_dir)


# Split dataset to train and val

In [ ]:
import os
import shutil
from random import shuffle
from sklearn.model_selection import train_test_split

def find_files(directory, extension=".jpg"):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(extension)]

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def copy_files(pairs, directory, label_directory):
    # Ensure target directories exist
    ensure_dir(directory)
    ensure_dir(label_directory)

    for img_path, label_path in pairs:
        img_name = os.path.basename(img_path)
        label_name = os.path.basename(label_path)

        shutil.copy2(img_path, os.path.join(directory, img_name))
        shutil.copy2(label_path, os.path.join(label_directory, label_name))
        print(f"Copied {img_path} to {os.path.join(directory, img_name)}")
        print(f"Copied {label_path} to {os.path.join(label_directory, label_name)}")

def split_data(images, labels, train_dir, val_dir, train_labels_dir, val_labels_dir, val_ratio=0.2):
    paired_data = list(zip(images, labels))
    train_pairs, val_pairs = train_test_split(paired_data, test_size=val_ratio, random_state=42)

    # Copy the files to train and validation directories
    copy_files(train_pairs, train_dir, train_labels_dir)
    copy_files(val_pairs, val_dir, val_labels_dir)

# Your directory paths
source_images_path = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images'
source_labels_path = '/content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels'
train_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train' #'/content/drive/MyDrive/SuperAI/Project/GDA/train/images'
val_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/val' #'/content/drive/MyDrive/SuperAI/Project/GDA/validation/images'
train_labels_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train' #'/content/drive/MyDrive/SuperAI/Project/GDA/train/labels'
val_labels_directory_path = '/content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/val' #'/content/drive/MyDrive/SuperAI/Project/GDA/validation/labels'

# Find all image and label files
image_files = find_files(source_images_path)
label_files = find_files(source_labels_path, extension=".txt")

# Split and copy the images and labels to train and validation directories
split_data(image_files, label_files, train_directory_path, val_directory_path,
           train_labels_directory_path, val_labels_directory_path, val_ratio=0.2)


Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0084.jpg to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0084.jpg
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels/0021.txt to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train/0021.txt
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0043.jpg to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0043.jpg
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels/0011.txt to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train/0011.txt
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0002.jpg to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train/0002.jpg
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_OBBlabels/0118.txt to /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/labels/train/0118.txt
Copied /content/drive/MyDrive/SuperAI/Project/GDA/train_temp_images/0010.

# write YAML file

In [ ]:
# Content to be written to the YAML file
yaml_content = """# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB # dataset root dir
train: /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/train # train images (relative to 'path')
val: /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/images/val # val images (relative to 'path')
#test: /content/drive/MyDrive/SuperAI/Project/GDA/test # test images (optional)

# Classes
names:
  0: Energy
  1: Sugar
  2: Fat
  3: Sodium
"""

# Write to the YAML file
with open('/content/drive/MyDrive/SuperAI/Project/GDA/GDA_yolo-obb.yaml', 'w') as file:
    file.write(yaml_content)

print('GDA_yolo-obb.yaml has been created.')


GDA_yolo-obb.yaml has been created.


# YOLO-OBB

In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 14.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Create a new YOLOv8n-OBB model from scratch
model = YOLO('yolov8n-obb.yaml')

# # Train the model on the GDA_yolo-obb.yaml dataset
results = model.train(data='/content/drive/MyDrive/SuperAI/Project/GDA/GDA_yolo-obb.yaml', epochs=100, imgsz=640)

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.yaml, data=/content/drive/MyDrive/SuperAI/Project/GDA/GDA_yolo-obb.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 755k/755k [00:00<00:00, 104MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 316MB/s]


AMP: checks passed ✅


...

    100/100      2.34G      1.089      1.809      2.206         28        640: 100%|██████████| 5/5 [00:01<00:00,  3.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.16s/it]

                   all         18         12      0.146       0.25      0.278      0.227



100 epochs completed in 0.236 hours.
Optimizer stripped from runs/obb/train2/weights/last.pt, 6.4MB
Optimizer stripped from runs/obb/train2/weights/best.pt, 6.4MB

Validating runs/obb/train2/weights/best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-obb summary (fused): 187 layers, 3077999 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


                   all         18         12      0.203      0.333      0.284      0.235
                Energy         18          3      0.116      0.333      0.125     0.0846
                 Sugar         18          3      0.354      0.333       0.22      0.174
                   Fat         18          3      0.239      0.333      0.393      0.335
                Sodium         18          3      0.103      0.333        0.4      0.347
Speed: 0.8ms preprocess, 6.0ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/obb/train2


---

In [ ]:
!cp -r /content/runs/obb/train2 /content/drive/MyDrive/SuperAI/Project/GDA/GDA-OBB/train2

๒

# OCR part

In [ ]:
! pip install -q pytesseract
! apt install -qq tesseract-ocr tesseract-ocr-tha

In [ ]:
import cv2
import numpy as np

# Load the template GDA image (the image of the GDA label you provided)
template = cv2.imread(image_files[0], 0)
w, h = template.shape[::-1]

# Load the target image where you want to find the template
target_image_path = 'path_to_target_image.png'
img_rgb = cv2.imread(target_image_path)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

# Apply template Matching
res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
threshold = 0.8  # You might need to adjust this threshold
loc = np.where(res >= threshold)

# If the template is found in the image
if loc[0].size > 0:
    pt = loc[::-1][0][0], loc[::-1][1][0]
    # Draw a rectangle around the matched region
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 255, 255), 2)

    # Crop the matched region from the target image
    roi = img_rgb[pt[1]:pt[1] + h, pt[0]:pt[0] + w]

    # Save or process the ROI...

# Show the result
cv2.imshow('Detected', img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import pytesseract
from pytesseract import Output

# Load the image
image = cv2.imread(image_files[0])

# Preprocess the image (if necessary)
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding
thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Use pytesseract to do OCR on the processed image
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, config=custom_config, lang='tha+eng')

# Print the text
print(text)
